ada app assessment

q3. In which year (2000 or 2015) did storms have a higher monetary impact within the boundaries of the 13 original colonies?

In [1]:
import pandas as pd
import numpy as np
storm2000 = pd.read_csv('StormEvents_details-ftp_v1.0_d2000_c20160223.csv')
storm2015 = pd.read_csv('StormEvents_details-ftp_v1.0_d2015_c20160810.csv')

In [2]:
print storm2000['DAMAGE_PROPERTY'].head() #sanity check
print storm2000['DAMAGE_PROPERTY'].tail()
print storm2015['DAMAGE_PROPERTY'].head()
print storm2015['DAMAGE_PROPERTY'].tail()

0    NaN
1    NaN
2    NaN
3    NaN
4     2K
Name: DAMAGE_PROPERTY, dtype: object
52002    NaN
52003    NaN
52004    NaN
52005    NaN
52006    NaN
Name: DAMAGE_PROPERTY, dtype: object
0    0.00K
1    0.00K
2    0.00K
3    0.00K
4    0.00K
Name: DAMAGE_PROPERTY, dtype: object
57516      0.00K
57517     39.00K
57518      0.00K
57519    500.00K
57520      0.00K
Name: DAMAGE_PROPERTY, dtype: object


In [3]:
#original thirteen colonies (copy-pasta'd from Wikipedia):
#Delaware, Pennsylvania, New Jersey, Georgia, Connecticut, Massachusetts Bay, Maryland, 
#South Carolina, New Hampshire, Virginia, New York, North Carolina, and Rhode Island

#if I wanted to use STATE = ...
thirteen = ['DELAWARE', 'PENNSYLVANIA', 'NEW JERSEY', 'GEORGIA', \
            'CONNECTICUT', 'MASSACHUSETTS', 'MARYLAND', 'SOUTH CAROLINA', \
            'NEW HAMPSHIRE', 'VIRGINIA', 'NEW YORK', 'NORTH CAROLINA', 'RHODE ISLAND']

#yeah, but do I want to be more accurate and go with whatever
#STATE_FIPS and CZ_FIPS lies within the Proclamation Line of 1763?
#
#
#... naaaaaaah.


def filterdf(df, feature, values):
    '''filters out out dataframe rows based on multiple values and appending them together'''
    df_new = pd.DataFrame()
    for thing in values:
        df_new = df_new.append(df[df[feature] == thing])
    return df_new
        
storm2000_13 = filterdf(storm2000, 'STATE', thirteen)
storm2015_13 = filterdf(storm2015, 'STATE', thirteen)

In [4]:
print storm2000_13['DAMAGE_PROPERTY'].head() #sanity check
print storm2000_13['DAMAGE_PROPERTY'].tail()
print storm2015_13['DAMAGE_PROPERTY'].head()
print storm2015_13['DAMAGE_PROPERTY'].tail()

129    NaN
182    NaN
226    NaN
227    NaN
335    NaN
Name: DAMAGE_PROPERTY, dtype: object
50895    NaN
50984    NaN
50985    NaN
51070    NaN
51071    NaN
Name: DAMAGE_PROPERTY, dtype: object
4332     0.00K
4333     0.00K
4334    30.00K
4583     0.00K
4584     0.00K
Name: DAMAGE_PROPERTY, dtype: object
42300    30.00K
45048     5.00K
45049    10.00K
45385     1.00K
47306     5.00K
Name: DAMAGE_PROPERTY, dtype: object


In [5]:
#DAMAGE_PROPERTY and DAMAGE_CROPS are strings that aren't traditionally formatted as an int or float
#so they need to be converted based on what the string has (K = 1000, M = 1000000, B = 1000000000)

#read the last char of a string to see if it's K, M, or B (while accounting for lack of a letter)
#multiply the rest of the string (which should be a number) by the letter's corresponding value
#create new features from these conversions and then sum them up

def convertformat(numberstring):
    '''converts individual strings to float equivalent'''
    if not numberstring: #empty string
        return 0
    elif numberstring[-1].upper() == 'K': #multiplier for one thousand
        return float(numberstring[0:-1]) * 1000
    elif numberstring[-1].upper() == 'M': #multiplier for one million
        return float(numberstring[0:-1]) * 1000000
    elif numberstring[-1].upper() == 'B': #multiplier for one billion
        return float(numberstring[0:-1]) * 1000000000
    else: #no multiplier
        return numberstring 

#test
print convertformat('0.3k')
print convertformat('3.42M')
print convertformat('23b')
print convertformat('0')
print convertformat('4')
print convertformat('')

SyntaxError: invalid syntax (<ipython-input-5-b6440ce88d51>, line 9)

In [ ]:
def convertcolumn(df, oldfeature, newfeature):
    '''runs values in column through convertformat and returns resulting new column'''
    df[oldfeature] = df[oldfeature].fillna(value='0')
    df[newfeature] = [convertformat(entry) for entry in df[oldfeature]]
    
#convertcolumn(storm2000_13, 'DAMAGE_PROPERTY', 'DP_new')

convertcolumn(storm2000_13, 'DAMAGE_PROPERTY', 'DP_new')
convertcolumn(storm2015_13, 'DAMAGE_PROPERTY', 'DP_new')
convertcolumn(storm2000_13, 'DAMAGE_CROPS', 'DC_new')
convertcolumn(storm2015_13, 'DAMAGE_CROPS', 'DC_new')

#storm2000_13['DP_new'] = [str(entry)+'2'  for entry in storm2000_13['DAMAGE_PROPERTY']]

#storm2000_13['DAMAGE_PROPERTY'] = storm2000_13['DAMAGE_PROPERTY'].fillna(value='0')
#storm2000_13['DP_new'] = [convertformat(entry) for entry in storm2000_13['DAMAGE_PROPERTY']]

In [ ]:
print storm2000_13['DP_new'].head() #sanity check
print storm2000_13['DP_new'].tail()
print storm2015_13['DP_new'].head()
print storm2015_13['DP_new'].tail()

In [ ]:
print pd.to_numeric(storm2000_13['DP_new'], errors='coerce').sum() + pd.to_numeric(storm2000_13['DC_new'], errors='coerce').sum()
print pd.to_numeric(storm2015_13['DP_new'], errors='coerce').sum() + pd.to_numeric(storm2015_13['DC_new'], errors='coerce').sum()
#the year 2000 experienced greater financial damage from storms than the year 2015